<a href="https://colab.research.google.com/github/codekali/Neural-SC-Desciptor/blob/master/firstTrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import json
from torch.utils.data import Dataset
dictionary={}
X,Y = [],[]
count=-1   
file=open("/content/drive/My Drive/Transformer/train.token.sbt")
for line in file:
  X.append(line)
  count+=1
count=0
file=open("/content/drive/My Drive/Transformer/train.token.nl")
for line in file:
  Y.append(line)
  count+=1
print(len(Y))
print(len(X))
X=X[0:15]
Y=Y[0:15]

445812
445812


In [2]:
print(len(Y))
print(len(X))

15
15


In [0]:
import torch
from torch.utils import data

class Dataset(data.Dataset):
  def __init__(self, X_item, Y_item):
    self.X_item=X_item
    self.Y_item=Y_item
    self.device=device

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.X_item)

  def __getitem__(self, index):
    X = self.X_item[index]
    Y = self.Y_item[index]
    return X, Y

In [4]:
#import mydata
import torch
from torch.utils import data
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
#cudann.benchmark = True

# Parameters
params = {'batch_size': 5,
          'shuffle': True,
          'num_workers': 6}
max_epochs = 10

training_set = Dataset(X,Y)
training_generator = data.DataLoader(training_set, **params)

cuda


In [5]:
!pip install transformers

     |████████████████████████████████| 573kB 12.4MB/s 
     |████████████████████████████████| 890kB 54.1MB/s 
     |████████████████████████████████| 3.7MB 49.4MB/s 
     |████████████████████████████████| 1.0MB 54.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=22bbf60f8cf35f54fc72fb6edf6c9599c1ece720ea25df929d70b41be084fa7d
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch.optim as optim

tokenizer = T5Tokenizer.from_pretrained('/content/drive/My Drive/Transformer/spiece.model')
model = T5ForConditionalGeneration.from_pretrained('t5-small')
model = model.to(device)
for epoch in range(2):
    loss=0
    # Training
    for local_batch, local_labels in training_generator: 
        """Transfer local_batch and labels to GPU"""
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)

        """Forward Function Implementation"""
        input_ids = tokenizer.batch_encode_plus(local_batch, return_tensors="pt",pad_to_max_length=True)
        label = tokenizer.batch_encode_plus(local_labels, return_tensors="pt",pad_to_max_length=True)
        outputs = model(input_ids=(input_ids['input_ids']).to(device), lm_labels=(label['input_ids']).to(device))
        loss, prediction_scores = outputs[:2]
        print(loss)
        """Forward Function Ends here"""
        """Loss and optimizer"""
        model.zero_grad()
        loss.backward()
        optimizer = optim.SGD(model.parameters(), lr=0.01)
        optimizer.zero_grad()
        optimizer.step()
    print("Done one epoch\n\n\n")
    print(loss)


Calling T5Tokenizer.from_pretrained() with the path to a single file or url is deprecated


AttributeError: ignored